In [ ]:
### 1. CONFIGURATION COLAB - GPU et montage Drive ###

print("="*80)
print("🚀 CONFIGURATION GOOGLE COLAB")
print("="*80)

# Vérifier qu'on est bien sur Colab
import sys
if 'google.colab' not in sys.modules:
    raise RuntimeError("❌ Ce notebook est conçu pour Google Colab uniquement !")

print("\n✓ Google Colab détecté")

# Vérifier le GPU
import tensorflow as tf
print(f"\n📊 Configuration matérielle :")
print(f"   TensorFlow version : {tf.__version__}")

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"   ✓ GPU disponible : {gpus[0].name}")
    # Configurer la mémoire GPU
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
else:
    print("   ⚠ Aucun GPU détecté - Allez dans Runtime > Change runtime type > GPU")

# Monter Google Drive
print("\n📁 Montage de Google Drive...")
from google.colab import drive
drive.mount('/content/drive')
print("   ✓ Google Drive monté")

print("\n✓ Configuration terminée !")

In [ ]:
### 2. ACCÈS AU DATASET SUR GOOGLE DRIVE ###

print("\n" + "="*80)
print("📁 ACCÈS AU DATASET SUR GOOGLE DRIVE")
print("="*80)

import os
from pathlib import Path

# Chemins Google Drive - ACCÈS DIRECT (pas de téléchargement)
DRIVE_BASE = Path('/content/drive/My Drive')

# Chercher le dataset dans différents emplacements possibles
possible_paths = [
    DRIVE_BASE / 'bird_dataset',
    DRIVE_BASE / 'data',
    DRIVE_BASE / 'projet_bird_man' / 'data',
    DRIVE_BASE / 'Colab Notebooks' / 'data',
    DRIVE_BASE / 'datasets' / 'bird_dataset',
]

dataset_root = None
print("\n🔍 Recherche du dataset dans Google Drive...")

for path in possible_paths:
    if path.exists() and (path / 'train_bird').exists():
        dataset_root = path
        print(f"   ✓ Dataset trouvé : {path}")
        break

# Si non trouvé, demander le chemin
if dataset_root is None:
    print("\n⚠ Dataset non trouvé automatiquement.")
    print("   Veuillez spécifier le chemin vers votre dataset :")
    print("   Exemple : /content/drive/My Drive/mon_dossier/data")
    
    # Vous pouvez modifier ce chemin manuellement
    CUSTOM_PATH = "/content/drive/My Drive/bird_dataset"  # ← MODIFIER ICI SI NÉCESSAIRE
    
    if Path(CUSTOM_PATH).exists():
        dataset_root = Path(CUSTOM_PATH)
        print(f"   ✓ Chemin personnalisé utilisé : {dataset_root}")
    else:
        print(f"\n❌ Chemin non trouvé : {CUSTOM_PATH}")
        print("   Vérifiez que le dataset est bien dans votre Google Drive")

# Définir les chemins train/valid
if dataset_root:
    TRAIN_PATH = dataset_root / 'train_bird'
    VALID_PATH = dataset_root / 'valid_bird'
    
    if TRAIN_PATH.exists() and VALID_PATH.exists():
        train_classes = len([d for d in TRAIN_PATH.iterdir() if d.is_dir()])
        valid_classes = len([d for d in VALID_PATH.iterdir() if d.is_dir()])
        print(f"\n✓ Dataset accessible directement sur Drive !")
        print(f"   📂 Chemin : {dataset_root}")
        print(f"   📊 Classes entraînement : {train_classes}")
        print(f"   📊 Classes validation : {valid_classes}")
    else:
        print(f"\n❌ Dossiers train_bird/valid_bird non trouvés dans {dataset_root}")
else:
    print("\n❌ Dataset non configuré. Modifiez CUSTOM_PATH ci-dessus.")

In [ ]:
### 3. IMPORTS ET CONFIGURATION ###

print("\n" + "="*80)
print("📦 IMPORTS ET CONFIGURATION")
print("="*80)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from PIL import Image
from io import BytesIO
import warnings
warnings.filterwarnings('ignore')

# TensorFlow / Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Configuration du style
plt.style.use('default')
sns.set_palette("husl")

# Paramètres globaux
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 0.001
MAX_IMAGES_PER_CLASS = 50  # Limite pour accélérer (mettre None pour tout utiliser)

print("\n✓ Imports terminés")
print(f"\n📊 Paramètres :")
print(f"   Taille images : {IMG_SIZE}x{IMG_SIZE}")
print(f"   Batch size : {BATCH_SIZE}")
print(f"   Epochs max : {EPOCHS}")
print(f"   Limite images/classe : {MAX_IMAGES_PER_CLASS or 'Aucune'}")

In [ ]:
### 4. ANALYSE DU DATASET - Créer un DataFrame ###

print("\n" + "="*80)
print("📊 ANALYSE DU DATASET")
print("="*80)

if dataset_root is None:
    print("\n⚠ Dataset non accessible")
    print("  Exécutez la cellule 2 d'abord et assurez-vous que le dataset est disponible")
    df_dataset = None
else:
    try:
        # Chemins des données (accès direct sur Drive)
        train_dir = dataset_root / 'train_bird'
        valid_dir = dataset_root / 'valid_bird'
        
        # Créer les listes de données
        data = []
        
        # Traiter les données d'entraînement
        print("\n✓ Analyse des données d'entraînement...")
        if train_dir.exists():
            for class_path in sorted(train_dir.iterdir()):
                if class_path.is_dir():
                    images = list(class_path.glob('*.[jJ][pP][gG]')) + \
                            list(class_path.glob('*.[jJ][pP][eE][gG]')) + \
                            list(class_path.glob('*.[pP][nN][gG]'))
                    
                    # Limiter à MAX_IMAGES_PER_CLASS images par classe
                    num_images = min(len(images), MAX_IMAGES_PER_CLASS) if MAX_IMAGES_PER_CLASS else len(images)
                    
                    data.append({
                        'Classe': class_path.name,
                        'Ensemble': 'Entraînement',
                        "Nombre d'images": num_images,
                        'Chemin': str(class_path)
                    })
        
        # Traiter les données de validation
        print("✓ Analyse des données de validation...")
        if valid_dir.exists():
            for class_path in sorted(valid_dir.iterdir()):
                if class_path.is_dir():
                    images = list(class_path.glob('*.[jJ][pP][gG]')) + \
                            list(class_path.glob('*.[jJ][pP][eE][gG]')) + \
                            list(class_path.glob('*.[pP][nN][gG]'))
                    
                    # Limiter à MAX_IMAGES_PER_CLASS images par classe
                    num_images = min(len(images), MAX_IMAGES_PER_CLASS) if MAX_IMAGES_PER_CLASS else len(images)
                    
                    data.append({
                        'Classe': class_path.name,
                        'Ensemble': 'Validation',
                        "Nombre d'images": num_images,
                        'Chemin': str(class_path)
                    })
        
        if data:
            # Créer le DataFrame
            df_dataset = pd.DataFrame(data)
            
            # Afficher les statistiques
            print("\n" + "-"*80)
            print("RÉSUMÉ DU DATASET")
            print("-"*80)
            
            n_classes = df_dataset['Classe'].nunique()
            total_images = df_dataset["Nombre d'images"].sum()
            
            print(f"\n📊 Statistiques globales :")
            print(f"   Nombre total de classes : {n_classes}")
            print(f"   Nombre total d'images : {total_images:,}")
            print(f"   Limite par classe : {MAX_IMAGES_PER_CLASS or 'Aucune'} images")
            
            print(f"\n📈 Répartition par ensemble :")
            stats = df_dataset.groupby('Ensemble').agg({
                'Classe': 'nunique',
                "Nombre d'images": ['sum', 'mean', 'min', 'max']
            })
            stats.columns = ['Nombre de classes', 'Total images', 'Moy/classe', 'Min', 'Max']
            print(stats.to_string())
            
            print(f"\n🏆 Top 5 classes par nombre d'images :")
            top_classes = df_dataset.nlargest(5, "Nombre d'images")[['Classe', 'Ensemble', "Nombre d'images"]]
            print(top_classes.to_string(index=False))
            
            print(f"\n✓ DataFrame créé avec succès !")
            print(f"   Forme : {df_dataset.shape}")
            
            # Afficher le DataFrame
            print(f"\n📋 Aperçu du DataFrame :")
            display(df_dataset.head(10))
        else:
            print("⚠ Aucune image trouvée dans le dataset")
            df_dataset = None
    
    except Exception as e:
        print(f"\n❌ Erreur lors de l'analyse : {e}")
        import traceback
        traceback.print_exc()
        df_dataset = None

In [ ]:
### 5. VISUALISATION DU DATASET ###

print("\n" + "="*80)
print("📊 VISUALISATION DU DATASET")
print("="*80)

if df_dataset is not None:
    # Visualisation de la distribution
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Distribution des images par classe (Entraînement)
    train_data = df_dataset[df_dataset['Ensemble'] == 'Entraînement'].sort_values("Nombre d'images", ascending=True)
    colors = plt.cm.viridis(np.linspace(0, 1, len(train_data)))
    axes[0].barh(train_data['Classe'], train_data["Nombre d'images"], color=colors)
    axes[0].set_xlabel("Nombre d'images", fontsize=12)
    axes[0].set_title('Distribution par classe (Entraînement)', fontsize=14, fontweight='bold')
    axes[0].grid(axis='x', alpha=0.3)
    
    # Pie chart de la répartition
    ensemble_counts = df_dataset.groupby('Ensemble')["Nombre d'images"].sum()
    axes[1].pie(ensemble_counts, labels=ensemble_counts.index, autopct='%1.1f%%', 
                startangle=90, colors=['#2ecc71', '#3498db'], explode=[0.02, 0.02])
    axes[1].set_title('Répartition Train/Validation', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print("\n✓ Visualisation terminée !")
else:
    print("⚠ DataFrame non disponible")

In [ ]:
### 6. EXEMPLES D'IMAGES PAR CLASSE ###

print("\n" + "="*80)
print("🖼️ EXEMPLES D'IMAGES PAR CLASSE")
print("="*80)

if dataset_root:
    # Afficher des exemples de chaque classe
    n_examples = 5
    classes_to_show = sorted([d.name for d in train_dir.iterdir() if d.is_dir()])[:n_examples]
    
    fig, axes = plt.subplots(n_examples, 3, figsize=(12, 3*n_examples))
    
    for idx, class_name in enumerate(classes_to_show):
        class_path = train_dir / class_name
        images = list(class_path.glob('*.jpg')) + list(class_path.glob('*.JPG')) + list(class_path.glob('*.jpeg'))
        
        for j in range(min(3, len(images))):
            img = Image.open(images[j])
            axes[idx, j].imshow(img)
            axes[idx, j].axis('off')
            if j == 0:
                axes[idx, j].set_ylabel(class_name.replace('-', '\n'), fontsize=9, rotation=0, ha='right', va='center')
    
    plt.suptitle('Exemples d\'images par classe', fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()
    
    print("\n✓ Exemples affichés !")
else:
    print("⚠ Dataset non disponible")

In [ ]:
### 7. PRÉPARATION DES DATA GENERATORS ###

print("\n" + "="*80)
print("⚙️ PRÉPARATION DES DONNÉES")
print("="*80)

if dataset_root is None:
    print("\n❌ Dataset non disponible")
    train_generator = None
    val_generator = None
    test_generator = None
else:
    # Data augmentation pour l'entraînement
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.2
    )
    
    val_datagen = ImageDataGenerator(rescale=1./255)
    
    print("\n✓ Création des generators depuis Google Drive...")
    
    # Generator d'entraînement (accès direct au Drive)
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='training',
        shuffle=True
    )
    
    # Generator de validation
    val_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation',
        shuffle=False
    )
    
    # Generator de test
    test_generator = val_datagen.flow_from_directory(
        valid_dir,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )
    
    # Infos
    class_names = list(train_generator.class_indices.keys())
    num_classes = len(class_names)
    
    print(f"\n📊 Résumé des données :")
    print(f"   Classes : {num_classes}")
    print(f"   Train batches : {len(train_generator)}")
    print(f"   Validation batches : {len(val_generator)}")
    print(f"   Test batches : {len(test_generator)}")
    
    print(f"\n📋 Liste des classes :")
    for i, name in enumerate(class_names):
        print(f"   {i}: {name}")
    
    print("\n✓ Données prêtes (accès direct depuis Drive) !")

In [ ]:
### 8. CRÉATION DU MODÈLE - Transfer Learning MobileNetV2 ###

print("\n" + "="*80)
print("🧠 CRÉATION DU MODÈLE")
print("="*80)

if train_generator is None:
    print("\n❌ Les data generators ne sont pas disponibles")
    print("   Exécutez la cellule 7 d'abord")
    model = None
else:
    print("\n✓ Chargement de MobileNetV2 pré-entraîné...")
    
    # Charger MobileNetV2 sans les couches de classification
    base_model = MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )
    
    # Geler les couches du modèle de base
    base_model.trainable = False
    
    print(f"   ✓ MobileNetV2 chargé : {len(base_model.layers)} couches")
    print(f"   ✓ Couches gelées pour transfer learning")
    
    # Construire le classificateur personnalisé
    print("\n✓ Construction du classificateur...")
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Compiler
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Résumé
    total_params = model.count_params()
    trainable_params = sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])
    
    print(f"\n📊 Architecture :")
    print(f"   Base : MobileNetV2 (ImageNet)")
    print(f"   Paramètres totaux : {total_params:,}")
    print(f"   Paramètres entraînables : {trainable_params:,}")
    print(f"   Classes de sortie : {num_classes}")
    
    print("\n✓ Modèle créé !")

In [ ]:
### 9. RÉSUMÉ DU MODÈLE ###

if model:
    model.summary()

In [ ]:
### 10. ENTRAÎNEMENT PHASE 1 - Classificateur ###

print("\n" + "="*80)
print("🚀 ENTRAÎNEMENT DU MODÈLE")
print("="*80)

if train_generator is None or model is None:
    print("\n❌ Erreur : Les données ou le modèle ne sont pas disponibles")
    history = None
else:
    import time
    
    # Callbacks
    callbacks_phase1 = [
        EarlyStopping(
            monitor='val_accuracy',
            patience=5,
            restore_best_weights=True,
            verbose=1,
            mode='max'
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=2,
            min_lr=1e-7,
            verbose=1
        ),
        ModelCheckpoint(
            '/content/best_model_phase1.h5',
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1,
            mode='max'
        )
    ]
    
    # ========== PHASE 1 : Entraîner le classificateur ==========
    print("\n" + "-"*60)
    print("📍 PHASE 1 : Entraînement du classificateur (couches gelées)")
    print("-"*60)
    
    start_time = time.time()
    
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=10,
        callbacks=callbacks_phase1,
        verbose=1
    )
    
    phase1_time = time.time() - start_time
    phase1_acc = max(history.history['val_accuracy'])
    
    print(f"\n✓ Phase 1 terminée en {phase1_time/60:.1f} min")
    print(f"   Meilleure précision : {phase1_acc*100:.2f}%")

In [ ]:
### 11. PHASE 2 : FINE-TUNING ###

print("\n" + "-"*60)
print("📍 PHASE 2 : Fine-tuning (dégel partiel)")
print("-"*60)

if history is None:
    print("\n❌ Phase 1 non terminée")
else:
    # Dégeler les 30 dernières couches
    base_model.trainable = True
    for layer in base_model.layers[:-30]:
        layer.trainable = False
    
    trainable_count = sum([1 for layer in model.layers if layer.trainable])
    print(f"   ✓ {trainable_count} couches maintenant entraînables")
    
    # Recompiler avec learning rate plus faible
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Nouveaux callbacks
    callbacks_phase2 = [
        EarlyStopping(
            monitor='val_accuracy',
            patience=5,
            restore_best_weights=True,
            verbose=1,
            mode='max'
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=2,
            min_lr=1e-8,
            verbose=1
        ),
        ModelCheckpoint(
            '/content/best_model_finetuned.h5',
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1,
            mode='max'
        )
    ]
    
    start_time = time.time()
    
    history_ft = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=10,
        callbacks=callbacks_phase2,
        verbose=1
    )
    
    phase2_time = time.time() - start_time
    
    # Combiner les historiques
    for key in history.history:
        history.history[key].extend(history_ft.history[key])
    
    final_acc = max(history_ft.history['val_accuracy'])
    
    print(f"\n✓ Phase 2 terminée en {phase2_time/60:.1f} min")
    print(f"   Meilleure précision : {final_acc*100:.2f}%")
    
    # Résumé final
    print("\n" + "="*60)
    print("📊 RÉSUMÉ DE L'ENTRAÎNEMENT")
    print("="*60)
    total_time = phase1_time + phase2_time
    print(f"   ⏱️ Temps total : {total_time/60:.1f} min")
    print(f"   📈 Précision Phase 1 : {phase1_acc*100:.2f}%")
    print(f"   🎯 Précision Phase 2 : {final_acc*100:.2f}%")
    print(f"   💾 Modèle sauvegardé : /content/best_model_finetuned.h5")

In [ ]:
### 12. ÉVALUATION SUR LE TEST SET ###

print("\n" + "="*80)
print("🎯 ÉVALUATION SUR LE TEST SET")
print("="*80)

# Évaluer
print("\n✓ Évaluation en cours...")
test_loss, test_accuracy = model.evaluate(test_generator, verbose=0)

print(f"\n📊 Résultats sur le test set :")
print(f"   Perte : {test_loss:.4f}")
print(f"   Précision : {test_accuracy*100:.2f}%")

# Matrice de confusion
from sklearn.metrics import confusion_matrix, classification_report

print("\n✓ Génération des prédictions...")
test_generator.reset()
predictions = model.predict(test_generator, verbose=0)
y_pred = np.argmax(predictions, axis=1)
y_true = test_generator.classes

# Rapport de classification
print("\n📋 Rapport de classification :")
print(classification_report(y_true, y_pred, target_names=class_names))

# Matrice de confusion
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(15, 12))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.title('Matrice de Confusion', fontsize=14, fontweight='bold')
plt.xlabel('Prédiction')
plt.ylabel('Vérité')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

print("\n✓ Évaluation terminée !")

In [ ]:
### 13. EXEMPLES DE PRÉDICTIONS ###

print("\n" + "="*80)
print("🖼️ EXEMPLES DE PRÉDICTIONS")
print("="*80)

# Récupérer quelques images
test_generator.reset()
test_images, test_labels = next(test_generator)

# Prédictions
preds = model.predict(test_images[:12], verbose=0)
pred_classes = np.argmax(preds, axis=1)
true_classes = np.argmax(test_labels[:12], axis=1)

# Affichage
fig, axes = plt.subplots(3, 4, figsize=(16, 12))
axes = axes.flatten()

for idx in range(12):
    img = (test_images[idx] * 255).astype(np.uint8)
    true_label = class_names[true_classes[idx]]
    pred_label = class_names[pred_classes[idx]]
    confidence = preds[idx][pred_classes[idx]] * 100
    
    axes[idx].imshow(img)
    
    correct = true_classes[idx] == pred_classes[idx]
    color = 'green' if correct else 'red'
    symbol = '✓' if correct else '✗'
    
    title = f'{symbol} {pred_label}\n({confidence:.1f}%)'
    axes[idx].set_title(title, color=color, fontsize=10, fontweight='bold')
    axes[idx].axis('off')

plt.suptitle('Exemples de prédictions (Vert=Correct, Rouge=Incorrect)', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("\n✓ Exemples affichés !")

In [ ]:
### 14. SAUVEGARDE DU MODÈLE SUR GOOGLE DRIVE ###

print("\n" + "="*80)
print("💾 SAUVEGARDE SUR GOOGLE DRIVE")
print("="*80)

if model is None:
    print("\n❌ Modèle non disponible")
else:
    # Créer le dossier de sauvegarde dans Drive
    save_dir = Path('/content/drive/My Drive/bird_classifier_model')
    save_dir.mkdir(parents=True, exist_ok=True)
    
    # Sauvegarder le modèle complet
    model_path = save_dir / 'bird_classifier_mobilenet.h5'
    model.save(str(model_path))
    print(f"\n✓ Modèle sauvegardé : {model_path}")
    
    # Sauvegarder les classes
    import json
    classes_path = save_dir / 'class_names.json'
    with open(classes_path, 'w') as f:
        json.dump(class_names, f, indent=2)
    print(f"✓ Classes sauvegardées : {classes_path}")
    
    # Sauvegarder l'historique
    if history:
        history_path = save_dir / 'training_history.json'
        # Convertir les valeurs numpy en float
        history_dict = {k: [float(v) for v in vals] for k, vals in history.history.items()}
        with open(history_path, 'w') as f:
            json.dump(history_dict, f, indent=2)
        print(f"✓ Historique sauvegardé : {history_path}")
    
    # Copier aussi le meilleur modèle
    import shutil
    if Path('/content/best_model_finetuned.h5').exists():
        shutil.copy('/content/best_model_finetuned.h5', save_dir / 'best_model_finetuned.h5')
        print(f"✓ Meilleur modèle copié")
    
    # Sauvegarder le DataFrame
    if df_dataset is not None:
        df_path = save_dir / 'dataset_info.csv'
        df_dataset.to_csv(df_path, index=False)
        print(f"✓ DataFrame sauvegardé : {df_path}")
    
    print(f"\n📁 Tous les fichiers sauvegardés dans :")
    print(f"   {save_dir}")
    print("\n✓ Sauvegarde terminée !")

In [ ]:
### 15. FONCTION DE PRÉDICTION ###

print("\n" + "="*80)
print("🔮 FONCTION DE PRÉDICTION")
print("="*80)

def predict_bird(image_path, model=model, class_names=class_names, top_k=3):
    """
    Prédit l'espèce d'oiseau à partir d'une image.
    
    Args:
        image_path: Chemin vers l'image
        model: Modèle entraîné
        class_names: Liste des noms de classes
        top_k: Nombre de prédictions à retourner
    
    Returns:
        dict: Résultats de la prédiction
    """
    # Charger et prétraiter l'image
    img = Image.open(image_path).convert('RGB')
    img_resized = img.resize((IMG_SIZE, IMG_SIZE))
    img_array = np.array(img_resized) / 255.0
    img_batch = np.expand_dims(img_array, axis=0)
    
    # Prédiction
    predictions = model.predict(img_batch, verbose=0)[0]
    
    # Top-K prédictions
    top_indices = np.argsort(predictions)[-top_k:][::-1]
    
    results = {
        'predictions': [
            {'class': class_names[i], 'confidence': float(predictions[i] * 100)}
            for i in top_indices
        ],
        'top_class': class_names[top_indices[0]],
        'confidence': float(predictions[top_indices[0]] * 100)
    }
    
    return results, img

# Test avec une image du dataset
print("\n✓ Fonction de prédiction créée")
print("\n📝 Exemple d'utilisation :")
print('   results, img = predict_bird("/chemin/vers/image.jpg")')
print('   print(f"Espèce: {results[\"top_class\"]} ({results[\"confidence\"]:.1f}%)")')

# Test réel avec une image du dataset sur Drive
if dataset_root and model:
    test_images_list = list(valid_dir.glob('*/*.jpg'))[:1]
    if test_images_list:
        test_img_path = test_images_list[0]
        results, img = predict_bird(test_img_path)
        
        print(f"\n🧪 Test avec : {test_img_path.name}")
        print(f"   Top 3 prédictions :")
        for pred in results['predictions']:
            print(f"     - {pred['class']}: {pred['confidence']:.1f}%")
        
        plt.figure(figsize=(6, 6))
        plt.imshow(img)
        plt.title(f"Prédiction: {results['top_class']}\n({results['confidence']:.1f}%)", 
                  fontsize=12, fontweight='bold')
        plt.axis('off')
        plt.show()

In [ ]:
### 16. UPLOAD ET PRÉDICTION D'IMAGES PERSONNELLES ###

print("\n" + "="*80)
print("📤 UPLOAD ET PRÉDICTION D'IMAGES")
print("="*80)

from google.colab import files

print("\n📌 Uploadez une image d'oiseau pour la classifier :")

try:
    uploaded = files.upload()
    
    for filename in uploaded.keys():
        print(f"\n🔍 Analyse de : {filename}")
        
        # Sauvegarder temporairement
        with open(f'/content/{filename}', 'wb') as f:
            f.write(uploaded[filename])
        
        # Prédiction
        results, img = predict_bird(f'/content/{filename}')
        
        # Affichage
        plt.figure(figsize=(10, 8))
        plt.imshow(img)
        
        title = f"Espèce prédite : {results['top_class']}\n"
        title += f"Confiance : {results['confidence']:.1f}%"
        plt.title(title, fontsize=14, fontweight='bold', color='green')
        plt.axis('off')
        plt.show()
        
        print(f"\n📊 Top 3 prédictions :")
        for i, pred in enumerate(results['predictions'], 1):
            bar = '█' * int(pred['confidence'] / 5)
            print(f"   {i}. {pred['class']}: {pred['confidence']:.1f}% {bar}")

except Exception as e:
    print(f"\n⚠ Aucun fichier uploadé ou erreur : {e}")
    print("   Vous pouvez réexécuter cette cellule pour uploader une image.")